# ***IMPORTING LIBRARIES AND MODULES***

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn import set_config

In [ ]:
pd.set_option('display.max_columns', None)


# ***GET DATAS***

In [ ]:
url = "https://drive.google.com/file/d/1NFHZhCOxgW1bu5q32OqVIVBDtSo2Alkh/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = df = pd.read_csv(path)

# ***SPLIT DATAS***

In [ ]:
# X and y creation
X = data.copy()
X.pop('Id')
y = X.pop("Expensive")
# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# ***PREPROCESSOR***

In [ ]:
# 1. defining categorical & ordinal columns
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

# 2. numerical pipeline
numeric_pipe = make_pipeline(SimpleImputer())

# 3. categorical pipeline
    # # 3.1 defining ordinal & onehot columns

ordinal_col_names = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'KitchenQual', 'FireplaceQu', 'LotShape',
       'BsmtFinType2', 'HeatingQC', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PoolQC', 'Fence']

ordinal_cols = X_cat.columns.get_indexer(ordinal_col_names)
ohe_cols = X_cat.columns.get_indexer(list(set(X_cat) - set(ordinal_col_names)))

X_cat_ordinal = X_cat.columns[ordinal_cols]
X_cat_ohe = X_cat.columns[ohe_cols]

    ## 3.2 explicitly determine categories for ordinal encoding including "N_A"
ExterQual_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
ExterCond_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
BsmtQual_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
BsmtCond_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
BsmtExposure_cats = ["N_A", "No", "Mn", "Av", "Gd"]
BsmtFinType1_cats = ["N_A", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"]
KitchenQual_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
FireplaceQu_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
LotShape_cats = ["N_A",'Reg', 'IR1', 'IR2', 'IR3']
BsmtFinType2_cats = ['N_A','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
HeatingQC_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
GarageFinish_cats = ['N_A','Unf','RFn','Fin']
GarageQual_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
GarageCond_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
PoolQC_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
Fence_cats = ["N_A",'NA','MnWw','GdWo','MnPrv','GdPrv']

cats_ord = [ExterQual_cats, ExterCond_cats, BsmtQual_cats, BsmtCond_cats, 
            BsmtExposure_cats, BsmtFinType1_cats, KitchenQual_cats, FireplaceQu_cats, 
            LotShape_cats,BsmtFinType2_cats,HeatingQC_cats,GarageFinish_cats,GarageQual_cats,
            GarageCond_cats,PoolQC_cats,Fence_cats]

        ### 3.2.2. defining the categorical encoder: a ColumnTransformer with 2 branches: ordinal & onehot
categorical_encoder = ColumnTransformer(
    transformers=[
        ("cat_ordinal", OrdinalEncoder(categories=cats_ord), ordinal_cols),
        ("cat_onehot", OneHotEncoder(handle_unknown="ignore"), ohe_cols),
    ]
)

    ## 3.3. categorical pipeline = "N_A" imputer + categorical encoder
categorical_pipe = make_pipeline(SimpleImputer(strategy="constant", fill_value="N_A"),
                                 categorical_encoder
                                )

# 4. full preprocessing: a ColumnTransformer with 2 branches: numeric & categorical
full_preprocessing = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        ("cat_pipe", categorical_pipe, X_cat.columns),
    ]
)


# ***MODELLING***

## *DecisionTreeClassifier* 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier


scaler = StandardScaler()

full_pipeline = make_pipeline(full_preprocessing, 
                              scaler,
                              DecisionTreeClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median","contant"],
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "decisiontreeclassifier__max_depth": range(2, 15, 2),
    "decisiontreeclassifier__min_samples_leaf": range(10, 100, 10),
    "decisiontreeclassifier__criterion": ["gini", "entropy"]
    

}

search = RandomizedSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      scoring='accuracy',
                      verbose=1, n_iter=100)

search.fit(X_train, y_train)

# create a dictionary to keep track of the scores of different models 
scores = {"dtree" : search.best_score_}


In [ ]:
dt_pred_test = search.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_dt = accuracy_score(y_true = y_test,
               y_pred = dt_pred_test
              )

scores["accuracy_dt"] = accuracy_dt

## *KNN*

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_full_pipeline = make_pipeline(full_preprocessing,
                                  scaler,
                                  KNeighborsClassifier()
                                 )

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median","contant"],
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "kneighborsclassifier__n_neighbors": range(2, 50),
    "kneighborsclassifier__weights": ["uniform", "distance"]
}

knn_search = RandomizedSearchCV(knn_full_pipeline,
                      param_grid,
                      cv=5,
                      scoring='accuracy',
                      verbose=1, n_iter=100)

knn_search.fit(X_train, y_train)

scores["knn"] = knn_search.best_score_

In [ ]:
knn_pred_test = knn_search.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_knn = accuracy_score(y_true = y_test,
               y_pred = knn_pred_test
              )

scores["accuracy_knn"] = accuracy_knn

## *Logistic Regression*

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV


scaler = StandardScaler()

LR_full_pipeline = make_pipeline(full_preprocessing,scaler, LogisticRegression())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median","contant"],
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "logisticregression__max_iter": range(1,100,10),
}

LR_search = RandomizedSearchCV(LR_full_pipeline,
                         param_grid,
                         cv=13,
                         scoring='accuracy',
                         verbose=1, n_iter=100)

LR_search.fit(X_train, y_train)

scores["LRegression"] = LR_search.best_score_

In [ ]:
LR_pred_test = LR_search.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_LR = accuracy_score(y_true = y_test,
               y_pred = LR_pred_test
              )

scores["accuracy_LR"] = accuracy_LR

## *SVC*

In [ ]:
from sklearn.svm import SVC

SVC_full_pipeline = make_pipeline(full_preprocessing,scaler, SVC())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median","contant"],
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "svc__gamma":['scale', 'auto'],
    "svc__max_iter":range(1,200,10),

}

SVC_search = RandomizedSearchCV(SVC_full_pipeline,
                          param_grid, 
                          cv=10, 
                          scoring="accuracy",
                          verbose=1, n_iter=100)

SVC_search.fit(X_train, y_train)
scores["SVC"] = SVC_search.best_score_

In [ ]:
SVC_pred_test = SVC_search.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_SVC = accuracy_score(y_true = y_test,
               y_pred = SVC_pred_test
              )

scores["accuracy_SVC"] = accuracy_SVC

## *Random Forest*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RandomForest_pipeline = make_pipeline(full_preprocessing, scaler, RandomForestClassifier())

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median","contant"],
    "standardscaler__with_mean": [True, False],
    "standardscaler__with_std": [True, False],
    "randomforestclassifier__n_estimators": range(10, 100, 20),
    "randomforestclassifier__max_depth": range(2, 15, 2),
    "randomforestclassifier__min_samples_leaf": range(10, 300, 50),
    "randomforestclassifier__criterion": ["gini", "entropy"]
}

RandomForest_search = RandomizedSearchCV(RandomForest_pipeline,
                                   param_grid,
                                   cv=5,
                                   scoring="accuracy",
                                   verbose=1, n_iter=100)

RandomForest_search.fit(X_train, y_train)

scores['RandomForest'] = RandomForest_search.best_score_

In [ ]:
RF_pred_test = RandomForest_search.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_RF = accuracy_score(y_true = y_test,
               y_pred = RF_pred_test
              )

scores["accuracy_RF"] = accuracy_RF

In [ ]:
scores

In [ ]:
url = " https://drive.google.com/file/d/15PfmTxmavQCT-f7iY9tgwWxm9t4GRees/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
Competition_DF = competition_df = pd.read_csv(path)

Compet_DF = Competition_DF.copy()
Compet_DF.pop('Id')

Alex_Compet_Submission = pd.DataFrame(Competition_DF["Id"])

LR_pred_sub = RandomForest_search.predict(Compet_DF)

Alex_Compet_Submission["Expensive"] = LR_pred_sub
Alex_Compet_Submission.head()
Alex_Compet_Submission.to_csv('Alex_Compet.csv', index=False)
from google.colab import files
files.download("Alex_Compet.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>